# 1 - create_table_creators_scrape_wiki

Este notebook realiza a ingestão das páginas da Wikipedia de criadores de conteúdo.  
O fluxo inclui:  
- leitura de um arquivo JSON/JSON.GZ parametrizado, com suporte a multiline;  
- aplicação de schema fixo (`wiki_page`) e validações de qualidade (nulos, vazios, duplicados);  
- escrita dos dados em tabela Delta gerenciada, com comentários de documentação em tabela e coluna.  

O resultado é a criação/atualização da tabela **default.creators_scrape_wiki**.


In [0]:
# Libraries and configuration

from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import logging, json, traceback

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


# Widgets (parameterization)

dbutils.widgets.text(
    "input_path",
    "/Volumes/workspace/default/gz_files/wiki_pages.json.gz",
    "File path (.json or .json.gz)"
)
dbutils.widgets.text(
    "output_table_name",
    "default.creators_scrape_wiki",
    "Target Delta table name"
)
dbutils.widgets.dropdown(
    "multiline",
    "false",
    ["false", "true"],
    "JSON multiline? (true/false)"
)

input_path        = dbutils.widgets.get("input_path").strip()
output_table_name = dbutils.widgets.get("output_table_name").strip()
use_multiline     = dbutils.widgets.get("multiline").lower() == "true"

logging.info(f"Arquivo de origem: {input_path}")
logging.info(f"Tabela destino: {output_table_name}")
logging.info(f"Leitura multiline: {use_multiline}")


# Input contract

schema = StructType([StructField("wiki_page", StringType(), True)])


# Path validation

try:
    _ = dbutils.fs.head(input_path, 1024)
except Exception as e:
    msg = f"ERRO: caminho inválido ou inacessível: {input_path} | {e}"
    logging.error(msg)
    dbutils.notebook.exit(msg)


# Ingestion

try:
    reader = spark.read.schema(schema)
    if use_multiline:
        reader = reader.option("multiLine", "true")
    creators_df = reader.json(input_path)
except Exception as e:
    logging.error("Falha ao ler o JSON de origem:\n" + traceback.format_exc())
    dbutils.notebook.exit(f"ERRO ao ler JSON: {e}")


# Cleaning and validation

creators_df = (
    creators_df
      .select(F.trim(F.col("wiki_page")).alias("wiki_page"))
      .filter(F.col("wiki_page").isNotNull() & (F.col("wiki_page") != ""))
      .dropDuplicates(["wiki_page"])
)

# empty check without using .isEmpty()
if creators_df.limit(1).count() == 0:
    dbutils.notebook.exit("ERRO: nenhum registro válido após limpeza.")


# Write (Delta)

try:
    (creators_df.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable(output_table_name)
    )

    spark.sql(f"""
    COMMENT ON TABLE {output_table_name}
    IS 'Tabela de páginas da Wikipedia (creators) — origem: {input_path}'
    """)

    spark.sql(f"""
    COMMENT ON COLUMN {output_table_name}.wiki_page
    IS 'Nome da página da Wikipedia associada a um criador de conteúdo'
    """)

except Exception as e:
    logging.error("Falha ao escrever na tabela Delta:\n" + traceback.format_exc())
    dbutils.notebook.exit(f"ERRO ao escrever Delta: {e}")


# Return

final_count = spark.table(output_table_name).count()
payload = {
    "status": "SUCESSO",
    "source_path": input_path,
    "target_table": output_table_name,
    "records_processed": final_count
}
logging.info(f"Processo finalizado. Registros ingeridos: {final_count}")
dbutils.notebook.exit(json.dumps(payload, ensure_ascii=False))
